<a href="https://colab.research.google.com/github/surabhi13gupta/LangChains/blob/main/pydantic_output_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install langchain_core
!pip install langchain_huggingface

In [4]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser

In [6]:
from pydantic import BaseModel

In [7]:
from pydantic import Field

In [3]:
llm = HuggingFaceEndpoint(
    repo_id="google/gemma-2-2b-it",
    task="text-generation"
)
model = ChatHuggingFace(llm=llm)

In [14]:
class Person(BaseModel):
    name : str=Field(description = "Name of the person")
    age : int=Field(description = "Age of the person", gt=18)
    city : str=Field(description = "Name of the city that person belongs to")

In [15]:
parser = PydanticOutputParser(pydantic_object=Person)

In [16]:
template = PromptTemplate(
    template='Generate the name, age and city of a fictional {place} person \n {format_instruction}',
    input_variables=['place'],
    partial_variables={'format_instruction':parser.get_format_instructions()}
)

In [17]:
chain = template | model | parser

In [18]:
result = chain.invoke({"place": "Indian"})

In [19]:
print(result)

name='Rohan Sharma' age=25 city=' Mumbai '


In [25]:
template.invoke({"place": "Indian"})

StringPromptValue(text='Generate the name, age and city of a fictional Indian person \n The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"description": "Name of the person", "title": "Name", "type": "string"}, "age": {"description": "Age of the person", "exclusiveMinimum": 18, "title": "Age", "type": "integer"}, "city": {"description": "Name of the city that person belongs to", "title": "City", "type": "string"}}, "required": ["name", "age", "city"]}\n```')